# Week 3 graded assignment

## Introduction

  This graded assignment is to explore, segment, and cluster the neighborhoods in the city of Toronto.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Transform the Online Data to a Dataframe</a>

2.  <a href="#item2">Get the Coordinates of the Neighborhoods</a>

3.  <a href="#item3">Explore and cluster the neighborhoods in Toronto</a>
    </font>
    </div>

## Table of Contents
Transform the Online Data to a Dataframe



1. [Transform the Online Data to a Dataframe](#1.Transform the Online Data to a Dataframe)
2. [Get the Coordinates of the Neighborhoods](#2.Get the Coordinates of the Neighborhoods)
3. [Explore and cluster the neighborhoods in Toronto](#3.Explore and cluster the neighborhoods in Toronto)

Import necessary libraries.

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from unicodedata import normalize # Return the normal form for the Unicode string

import json 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


<a id='item1'></a>


## 1. Transform the Online Data to a Dataframe


Use pandas to transform the data in the table on the Wikipedia page:https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M into a pandas dataframe.


In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
table=pd.read_html(url)[0]

In [3]:
len(table)

180

In [4]:
table[0:5]

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Tranform the data into a _pandas_ dataframe


In [55]:
neighborhoods_data = pd.DataFrame(table)
neighborhoods_data.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Preprocessing the Data

In [81]:
# drop the borough not assigned
neighborhoods=neighborhoods_data[neighborhoods_data.Borough !="Not assigned"]

In [82]:
# reset the index 
neighborhoods.reset_index(drop=True,inplace=True)

In [86]:
neighborhoods.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [87]:
neighborhoods.shape

(103, 3)

In [88]:
print('The dataframe has {} boroughs and {} Postal Codes.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 Postal Codes.


## 2. Get the Coordinates of the Neighborhoods

In [89]:
#!conda install -c conda-forge geocoder --yes 

In [90]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="goluck8@hotmail.com")
location = geolocator.geocode(address)
latitude_Toronto = location.latitude
longitude_Toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_Toronto, longitude_Toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [91]:
#!pip install pgeocode

In [92]:
import pgeocode

In [93]:
#define a function to find the latitude and lontitude from a zipcode
def zipcode_to_coordinate(zipcode):
    nomi = pgeocode.Nominatim('ca')
    location = nomi.query_postal_code(zipcode)
    return location.latitude,location.longitude

In [94]:
#call the function to get latitude&longitude for each postal code 
lat,lon=[],[]
for i in range(len(neighborhoods["Postal Code"])):
    a,b=zipcode_to_coordinate(neighborhoods["Postal Code"][i])
    lat.append(a)
    lon.append(b)
lat[0:5],lon[0:5]

([43.7545, 43.7276, 43.6555, 43.7223, 43.6641],
 [-79.33, -79.3148, -79.3626, -79.4504, -79.3889])

In [95]:
#add latitude$longitude to 2 new columns
neighborhoods.loc[:,'Latitude']=lat
neighborhoods.loc[:,'Longitude']=lon

C:\Users\EDWAR\anaconda3\lib\site-packages\pandas\core\indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\EDWAR\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [96]:
neighborhoods.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889


## 3. Explore and cluster the neighborhoods in Toronto


#### Create an area with only boroughs that contain the word "Toronto "


In [100]:
#creat a new DataFrame from scratch
columns=["Postal Code","Borough","Neighbourhood","Latitude","Longitude"]
Toronto_data=pd.DataFrame(columns=columns)
Toronto_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude


In [101]:
#drop boroughs without "Toronto" in the name
for i in range(len(neighborhoods["Postal Code"])):
    if "Toronto" in neighborhoods.loc[i,"Borough"]:
        Toronto_data.loc[i]=neighborhoods.loc[i]    
#reset index
Toronto_data.reset_index(drop=True,inplace=True)
Toronto_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,M4E,East Toronto,The Beaches,43.6784,-79.2941


In [102]:
Toronto_data.shape

(39, 5)

#### Create a map of Toronto with neighborhoods superimposed on top.

**Folium** is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.


In [103]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude_Toronto, longitude_Toronto], zoom_start=10)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

#### Define Foursquare Credentials and Version


In [104]:
CLIENT_ID = 'QO5ISBHFPQLGZDKTPQHZXQM3APJLN3GJUDHWTOXIOIUZUUEC' # your Foursquare ID
CLIENT_SECRET = 'X5WI4XZJG2GLGCS3ZWREHXAO5XTGTFANO5B0PWNYNADAMZRH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QO5ISBHFPQLGZDKTPQHZXQM3APJLN3GJUDHWTOXIOIUZUUEC
CLIENT_SECRET:X5WI4XZJG2GLGCS3ZWREHXAO5XTGTFANO5B0PWNYNADAMZRH


#### Let's explore the first neighborhood in our dataframe.


Get the neighborhood's name.


In [105]:
Toronto_data.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

Get the neighborhood's latitude and longitude values.


In [107]:
neighborhood_latitude = Toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name =Toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6555, -79.3626.


#### Now, let's get the top 100 venues that are in Regent Park and Harbourfront within a radius of 500 meters.


First, let's create the GET request URL. Name your URL **url**.


In [108]:
# type your answer here
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude,VERSION, 500, LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?client_id=QO5ISBHFPQLGZDKTPQHZXQM3APJLN3GJUDHWTOXIOIUZUUEC&client_secret=X5WI4XZJG2GLGCS3ZWREHXAO5XTGTFANO5B0PWNYNADAMZRH&ll=43.6555,-79.3626&v=20180605&radius=500&limit=100'

Double-click **here** for the solution.

<!-- The correct answer is:
LIMIT = 100 # limit of number of venues returned by Foursquare API
-->

<!--
radius = 500 # define radius
-->

<!--
\\\\ # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL
--> 


Send the GET request and examine the resutls


In [109]:
results = requests.get(url).json()

From the Foursquare lab in the previous module, we know that all the information is in the _items_ key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.


In [158]:
venues = results['response']['groups'][0]['items']

Now we are ready to clean the json and structure it into a _pandas_ dataframe.


In [159]:
nearby_venues = pd.json_normalize(venues) # flatten JSON
nearby_venues.head(3)

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.neighborhood,venue.venuePage.id
0,e-0-54ea41ad498e9a11e9e13308-0,0,"[{'summary': 'This spot is popular', 'type': '...",54ea41ad498e9a11e9e13308,Roselle Desserts,362 King St E,Trinity St,43.653447,-79.362017,"[{'label': 'display', 'lat': 43.65344672305267...",233,M5A 1K9,CA,Toronto,ON,Canada,"[362 King St E (Trinity St), Toronto ON M5A 1K...","[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",0,[],NaN,NaN
1,e-0-53b8466a498e83df908c3f21-1,0,"[{'summary': 'This spot is popular', 'type': '...",53b8466a498e83df908c3f21,Tandem Coffee,368 King St E,at Trinity St,43.653559,-79.361809,"[{'label': 'display', 'lat': 43.65355870959944...",225,NaN,CA,Toronto,ON,Canada,"[368 King St E (at Trinity St), Toronto ON, Ca...","[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",0,[],NaN,NaN
2,e-0-4af59046f964a520e0f921e3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4af59046f964a520e0f921e3,Figs Breakfast & Lunch,344 Queen St. E.,at Parliament St.,43.655675,-79.364503,"[{'label': 'display', 'lat': 43.65567455427388...",154,M5A 1S8,CA,Toronto,ON,Canada,"[344 Queen St. E. (at Parliament St.), Toronto...","[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",0,[],NaN,NaN


In [160]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Roselle Desserts,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",43.653447,-79.362017
1,Tandem Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.653559,-79.361809
2,Figs Breakfast & Lunch,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",43.655675,-79.364503
3,The Yoga Lounge,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",43.655515,-79.364955
4,Body Blitz Spa East,"[{'id': '4bf58dd8d48988d1ed941735', 'name': 'S...",43.654735,-79.359874
5,Sumach Espresso,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.658135,-79.359515
6,Morning Glory Cafe,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",43.653947,-79.361149
7,Berkeley Church,"[{'id': '4bf58dd8d48988d171941735', 'name': 'E...",43.655123,-79.365873
8,Sukhothai,"[{'id': '4bf58dd8d48988d149941735', 'name': 'T...",43.658444,-79.365681
9,Rooster Coffee,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.651900,-79.365609


In [161]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [162]:
# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,The Yoga Lounge,Yoga Studio,43.655515,-79.364955
4,Body Blitz Spa East,Spa,43.654735,-79.359874
5,Sumach Espresso,Coffee Shop,43.658135,-79.359515
6,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
7,Berkeley Church,Event Space,43.655123,-79.365873
8,Sukhothai,Thai Restaurant,43.658444,-79.365681
9,Rooster Coffee,Coffee Shop,43.651900,-79.365609


In [163]:
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,The Yoga Lounge,Yoga Studio,43.655515,-79.364955
4,Body Blitz Spa East,Spa,43.654735,-79.359874
5,Sumach Espresso,Coffee Shop,43.658135,-79.359515
6,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
7,Berkeley Church,Event Space,43.655123,-79.365873
8,Sukhothai,Thai Restaurant,43.658444,-79.365681
9,Rooster Coffee,Coffee Shop,43.651900,-79.365609


And how many venues were returned by Foursquare?


In [164]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

23 venues were returned by Foursquare.


<a id='item2'></a>


### Explore all Neighborhoods in Toronto


#### Let's create a function to repeat the same process to all the neighborhoods in Toronto


In [190]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])# item in venue_list which is in venues_list
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called Toronto_venues_.


In [191]:
# type your answer here
Toronto_venues=getNearbyVenues(Toronto_data['Neighbourhood'], Toronto_data['Latitude'], Toronto_data['Longitude'], radius=500)

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

#### Let's check the size of the resulting dataframe


In [192]:
print(Toronto_venues.shape)
Toronto_venues.head()

(1522, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.6555,-79.3626,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.6555,-79.3626,Body Blitz Spa East,43.654735,-79.359874,Spa


Let's check how many venues were returned for each neighborhood


In [193]:
Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,91,91,91,91,91,91
"Brockton, Parkdale Village, Exhibition Place",38,38,38,38,38,38
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",13,13,13,13,13,13
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",56,56,56,56,56,56
Central Bay Street,56,56,56,56,56,56
Christie,13,13,13,13,13,13
Church and Wellesley,78,78,78,78,78,78
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,22,22,22,22,22,22


#### Let's find out how many unique categories can be curated from all the returned venues


In [194]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 217 uniques categories.


<a id='item3'></a>


In [195]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe 
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighbourhood,Accessories Store,Adult Boutique,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.


In [196]:
Toronto_onehot.shape

(1522, 218)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [197]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean()
Toronto_grouped

,Accessories Store,Adult Boutique,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
Neighbourhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Berczy Park,0.000000,0.000000,0.010989,0.021978,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010989,0.043956,0.000000,0.000000,0.000000,0.010989,0.000000,0.032967,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.021978,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.043956,0.000000,0.000000,0.021978,0.000000,0.000000,0.010989,0.032967,0.098901,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010989,0.000000,0.010989,0.0,0.000000,0.000000,0.000000,0.010989,0.000000,0.000000,0.000000,0.000000,0.021978,0.010989,0.000000,0.010989,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010989,0.000000,0.000000,0.000

In [198]:
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped

,Neighbourhood,Accessories Store,Adult Boutique,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Stadium,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Cafeteria,College Gym,College Rec Center,College Theater,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.010989,0.021978,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010989,0.043956,0.000000,0.000000,0.000000,0.010989,0.000000,0.032967,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.021978,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.043956,0.000000,0.000000,0.021978,0.000000,0.000000,0.010989,0.032967,0.098901,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010989,0.000000,0.010989,0.0,0.000000,0.000000,0.000000,0.010989,0.000000,0.000000,0.000000,0.000000,0.021978,0.010989,0.000000,0.010989,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.010989,0.000000,0.000000,0.000000,0.000000,0.021978,0.010989,0.000000,0.010989,0.000000,0.000000,0.00,0.010989,0.010989,0.010989,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.010989,0.000000,0.000000,0.00,0.000000,0.00,0.010989,0.010989,0

#### Let's confirm the new size


In [199]:
Toronto_grouped.shape

(39, 218)

#### Let's print each neighborhood along with the top 5 most common venues


In [200]:
temp1=Toronto_grouped[Toronto_grouped["Neighbourhood"]=="Lawrence Park"].T.reset_index()# transpose is crucial
temp1.head()

,index,18
0,Neighbourhood,Lawrence Park
1,Accessories Store,0
2,Adult Boutique,0
3,American Restaurant,0
4,Art Gallery,0


In [201]:
temp1.columns = ['venue','freq']
temp1 = temp1.iloc[1:] # first line is the names
temp1.head()

,venue,freq
1,Accessories Store,0
2,Adult Boutique,0
3,American Restaurant,0
4,Art Gallery,0
5,Art Museum,0


In [202]:
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.10
1               Hotel  0.05
2                Café  0.04
3  Seafood Restaurant  0.04
4              Bakery  0.04


----Brockton, Parkdale Village, Exhibition Place----
                    venue  freq
0                    Café  0.08
1             Coffee Shop  0.08
2  Thrift / Vintage Store  0.05
3               Gift Shop  0.05
4          Breakfast Spot  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                 venue  freq
0           Restaurant  0.15
1          Coffee Shop  0.15
2          Yoga Studio  0.08
3  Japanese Restaurant  0.08
4     Sushi Restaurant  0.08


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0         Coffee Shop  0.07
1  Italian Restaurant  0.05
2                Café  0.05
3                 Bar  0.05
4          Restaurant  0.04


----Centr

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [203]:
#find top n venues in each row(neighborhood)
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [204]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd'] # 1st,2nd,3rd

# create columns according to number of top venues
columns = ['Neighbourhood'] # column name
#add another 10 columns
for ind in np.arange(num_top_venues): 
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):  # 40X332, 40 neighborhoods
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(40)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Hotel,Café,Bakery,Seafood Restaurant,Cocktail Bar,Beer Bar,Restaurant,Japanese Restaurant,Cheese Shop
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Thrift / Vintage Store,Gift Shop,Breakfast Spot,Italian Restaurant,Pizza Place,Brewery,Gym,Sandwich Place
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Restaurant,Yoga Studio,Intersection,Deli / Bodega,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Bank,Breakfast Spot
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Café,Italian Restaurant,Bar,Restaurant,Grocery Store,Gym / Fitness Center,French Restaurant,Speakeasy,Park
4,Central Bay Street,Coffee Shop,Pizza Place,Italian Restaurant,Café,Sandwich Place,Middle Eastern Restaurant,Bubble Tea Shop,Burger Joint,Shopping Mall,Spa
5,Christie,Café,Grocery Store,Athletics & Sports,Candy Store,Coffee Shop,Bank,Baby Store,Playground,Park,Discount Store
6,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Fast Food Restaurant,Restaurant,Hotel,Mediterranean Restaurant,Men's Store,Dance Studio
7,"Commerce Court, Victoria Hotel",Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Gym,Seafood Restaurant,Salad Place,American Restaurant,Steakhouse
8,Davisville,Italian Restaurant,Dessert Shop,Sandwich Place,Coffee Shop,Café,Seafood Restaurant,Gym,Diner,Toy / Game Store,Farmers Market
9,Davisville North,Food & Drink Shop,Breakfast Spot,Department Store,Park,Gym / Fitness Center,Dog Run,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop


<a id='item4'></a>


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [205]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1) # 1 means column

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:40] 

array([3, 3, 3, 3, 3, 0, 3, 3, 3, 0, 0, 3, 2, 3, 0, 1, 3, 3, 4, 3, 0, 0,
       3, 3, 3, 3, 0, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [206]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_) #insert in 0(first) column
neighborhoods_venues_sorted

,Cluster Labels,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,3,Berczy Park,Coffee Shop,Hotel,Café,Bakery,Seafood Restaurant,Cocktail Bar,Beer Bar,Restaurant,Japanese Restaurant,Cheese Shop
1,3,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Thrift / Vintage Store,Gift Shop,Breakfast Spot,Italian Restaurant,Pizza Place,Brewery,Gym,Sandwich Place
2,3,"Business reply mail Processing Centre, South C...",Coffee Shop,Restaurant,Yoga Studio,Intersection,Deli / Bodega,Sushi Restaurant,Japanese Restaurant,Italian Restaurant,Bank,Breakfast Spot
3,3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Café,Italian Restaurant,Bar,Restaurant,Grocery Store,Gym / Fitness Center,French Restaurant,Speakeasy,Park
4,3,Central Bay Street,Coffee Shop,Pizza Place,Italian Restaurant,Café,Sandwich Place,Middle Eastern Restaurant,Bubble Tea Shop,Burger Joint,Shopping Mall,Spa
5,0,Christie,Café,Grocery Store,Athletics & Sports,Candy Store,Coffee Shop,Bank,Baby Store,Playground,Park,Discount Store
6,3,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Fast Food Restaurant,Restaurant,Hotel,Mediterranean Restaurant,Men's Store,Dance Studio
7,3,"Commerce Court, Victoria Hotel",Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Gym,Seafood Restaurant,Salad Place,American Restaurant,Steakhouse
8,3,Davisville,Italian Restaurant,Dessert Shop,Sandwich Place,Coffee Shop,Café,Seafood Restaurant,Gym,Diner,Toy / Game Store,Farmers Market
9,0,Davisville North,Food & Drink Shop,Breakfast Spot,Department Store,Park,Gym / Fitness Center,Dog Run,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop


In [207]:
Toronto_merged = Toronto_data

# merge Toronto_grouped with Toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,3,Coffee Shop,Breakfast Spot,Theater,Distribution Center,Pub,Electronics Store,Restaurant,Event Space,Food Truck,Spa
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,3,Coffee Shop,Sushi Restaurant,Dance Studio,Italian Restaurant,Mexican Restaurant,Café,Burrito Place,College Cafeteria,Bubble Tea Shop,Ramen Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,3,Coffee Shop,Clothing Store,Japanese Restaurant,Italian Restaurant,Hotel,Café,Middle Eastern Restaurant,Cosmetics Shop,Pizza Place,Fast Food Restaurant
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,3,Coffee Shop,Seafood Restaurant,Café,Gastropub,American Restaurant,Italian Restaurant,Cocktail Bar,Hotel,Restaurant,Beer Bar
4,M4E,East Toronto,The Beaches,43.6784,-79.2941,3,Neighborhood,Cheese Shop,Coffee Shop,Bakery,Pub,Trail,Gastropub,Health Food Store,Falafel Restaurant,Dumpling Restaurant


Finally, let's visualize the resulting clusters


In [208]:
# create map
map_clusters = folium.Map(location=[latitude_Toronto, longitude_Toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1


In [209]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]] # column1, 5~end

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,0,Café,Grocery Store,Athletics & Sports,Candy Store,Coffee Shop,Bank,Baby Store,Playground,Park,Discount Store
9,West Toronto,0,Park,Bakery,Grocery Store,Pharmacy,Pizza Place,Pool,Middle Eastern Restaurant,Café,Smoke Shop,Liquor Store
10,Downtown Toronto,0,Harbor / Marina,Café,Park,Music Venue,Yoga Studio,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
20,Central Toronto,0,Food & Drink Shop,Breakfast Spot,Department Store,Park,Gym / Fitness Center,Dog Run,Eastern European Restaurant,Flower Shop,Fish Market,Fish & Chips Shop
23,Central Toronto,0,Playground,Gym Pool,Park,Garden,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
29,Central Toronto,0,Park,Gym,Grocery Store,Thai Restaurant,Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
33,Downtown Toronto,0,Playground,Candy Store,Park,Grocery Store,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


#### Cluster 2


In [210]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,West Toronto,1,Park,Residential Building (Apartment / Condo),Yoga Studio,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


#### Cluster 3


In [211]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,2,Home Service,Dumpling Restaurant,Food & Drink Shop,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
21,Central Toronto,2,Home Service,Construction & Landscaping,Park,Trail,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


#### Cluster 4


In [212]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,3,Coffee Shop,Breakfast Spot,Theater,Distribution Center,Pub,Electronics Store,Restaurant,Event Space,Food Truck,Spa
1,Downtown Toronto,3,Coffee Shop,Sushi Restaurant,Dance Studio,Italian Restaurant,Mexican Restaurant,Café,Burrito Place,College Cafeteria,Bubble Tea Shop,Ramen Restaurant
2,Downtown Toronto,3,Coffee Shop,Clothing Store,Japanese Restaurant,Italian Restaurant,Hotel,Café,Middle Eastern Restaurant,Cosmetics Shop,Pizza Place,Fast Food Restaurant
3,Downtown Toronto,3,Coffee Shop,Seafood Restaurant,Café,Gastropub,American Restaurant,Italian Restaurant,Cocktail Bar,Hotel,Restaurant,Beer Bar
4,East Toronto,3,Neighborhood,Cheese Shop,Coffee Shop,Bakery,Pub,Trail,Gastropub,Health Food Store,Falafel Restaurant,Dumpling Restaurant
5,Downtown Toronto,3,Coffee Shop,Hotel,Café,Bakery,Seafood Restaurant,Cocktail Bar,Beer Bar,Restaurant,Japanese Restaurant,Cheese Shop
6,Downtown Toronto,3,Coffee Shop,Pizza Place,Italian Restaurant,Café,Sandwich Place,Middle Eastern Restaurant,Bubble Tea Shop,Burger Joint,Shopping Mall,Spa
8,Downtown Toronto,3,Café,Coffee Shop,Hotel,Gym,Restaurant,Asian Restaurant,Steakhouse,Sushi Restaurant,Thai Restaurant,Salad Place
11,West Toronto,3,Bar,Coffee Shop,Cocktail Bar,Vegetarian / Vegan Restaurant,Asian Restaurant,Restaurant,Brewery,Italian Restaurant,Japanese Restaurant,Seafood Restaurant
12,East Toronto,3,Greek Restaurant,Ice Cream Shop,Café,Italian Restaurant,Coffee Shop,Yoga Studio,Dessert Shop,Bubble Tea Shop,Cocktail Bar,Restaurant


#### Cluster 5


In [213]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,4,Photography Studio,Park,Yoga Studio,Dumpling Restaurant,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant
